In [1]:
# import cv2
# import imageio.v3 as iio
# import numpy as np
# import read_roi 
# from read_roi import read_roi_file
# import os

# data_dir = "/media/molloi-lab/System Files/BAC_dataset/Clean_Dataset_full"

# def run():
#     for foldername, subfolders, filenames in os.walk(data_dir):
#         for filename in filenames:
#             if filename.endswith("roi"):
#                 curr_f_name = os.path.splitext(filename)[0]
#                 png_file_name = curr_f_name + ".png"
#                 png_path = os.path.join(foldername, png_file_name)
#                 roi_path = os.path.join(foldername, filename)
#                 if not os.path.isfile(png_path):
#                     print("Not found: "+png_path)
#                 else:
#                     # read png
#                     lbl = iio.imread(png_path)
#                     # read roivsc
#                     roi = read_roi_file(roi_path)[curr_f_name]
#                     if "paths" in roi.keys():
#                         # read "paths"
#                         paths = roi["paths"]
#                         for ps in paths:
#                             pts = np.array(ps, np.int32)
#                             cv2.fillPoly(lbl, [pts], 255) # 255 stands for white color
#                         cv2.imwrite(png_path, lbl)
#                     else:
#                         # read "x" and "y"
#                         x = roi["x"]
#                         y = roi["y"]
#                         pts = np.array(list(zip(x, y)), np.int32)
#                         cv2.fillPoly(lbl, [pts], 255) # 255 stands for white color
#                         cv2.imwrite(png_path, lbl)
#                 print(png_path)

# run()


In [6]:
import cv2
import imageio.v3 as iio
import numpy as np
import read_roi
from read_roi import read_roi_file
import os
import threading

data_dir = "/media/molloi-lab/2TB/Clean_Dataset_full"

# Define the function that will be executed by each thread
def process_file(roi_path, png_path):
    # read png
    lbl = iio.imread(png_path)
    # read roivsc
    curr_f_name = os.path.splitext(os.path.basename(roi_path))[0]
    roi = read_roi_file(roi_path)[curr_f_name]
    if "paths" in roi.keys():
        # read "paths"
        paths = roi["paths"]
        for ps in paths:
            pts = np.array(ps, np.int32)
            cv2.fillPoly(lbl, [pts], 255) # 255 stands for white color
        cv2.imwrite(png_path, lbl)
    elif "x" in roi.keys():
        # read "x" and "y"
        x = roi['x']
        y = roi['y']
        pts = np.array(list(zip(x, y)), np.int32)
        cv2.fillPoly(lbl, [pts], 255) # 255 stands for white color
        cv2.imwrite(png_path, lbl)
    else:
        print()
        print(roi_path)
        print(roi.keys())
        print()

def run():
    threads = []
    max_threads = 64  # You can tune this number
    for foldername, subfolders, filenames in os.walk(data_dir):
        for filename in filenames:
            if filename.endswith("roi"):
                curr_f_name = os.path.splitext(filename)[0]
                png_file_name = curr_f_name + ".png"
                png_path = os.path.join(foldername, png_file_name)
                roi_path = os.path.join(foldername, filename)
                if not os.path.isfile(png_path):
                    print("Not found: "+png_path)
                else:
                    # Create a new thread for each file and start it
                    t = threading.Thread(target=process_file, args=(roi_path, png_path))
                    t.start()
                    threads.append(t)
                    
                    # Wait for all threads to finish before creating new ones if max_threads reached
                    if len(threads) >= max_threads:
                        for t in threads:
                            t.join()
                        threads = []

    # Wait for any remaining threads to finish
    for t in threads:
        t.join()

In [8]:
run() # run time is ~21s


/media/molloi-lab/2TB/Clean_Dataset_full/SID-114203/L_MLO.2294_1914.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])


/media/molloi-lab/2TB/Clean_Dataset_full/SID-133482/R_MLO.3328_2560.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])


/media/molloi-lab/2TB/Clean_Dataset_full/SID-137291/R_CC.4096_3328.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])


/media/molloi-lab/2TB/Clean_Dataset_full/SID-137421/R_CC.4096_3328.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])


/media/molloi-lab/2TB/Clean_Dataset_full/SID-137421/R_MLO.4096_3328.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])


/media/molloi-lab/2TB/Clean_Dataset_full/SID-115429/L_CC.3328_2560.roi
dict_keys(['type', 'left', 'top', 'width', 'height', 'arc_size', 'name', 'position'])

